<a href="https://colab.research.google.com/github/benjaminbrown038/Hugging-Face/blob/main/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hugging Face Image Classification Notebook


[Imports](#imports)

[Data](#data)

##### Imports 
<a name="imports"></a>


In [ ]:
%%capture
!pip3 install transformers datasets evaluate
import tensorflow as tf
from tensorflow import keras
import torch
import evaluate
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.keras import layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

from huggingface_hub import notebook_login
from transformers import AutoImageProcessor, DefaultDataCollator, AutoModelForImageClassification, TrainingArguments, Trainer, create_optimizer, TFAutoModelForImageClassification, pipeline
from datasets import load_dataset
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback

#notebook_login()

#### Food Dataset

##### Data 
<a name="data"></a>


In [ ]:
#%%capture
food = load_dataset("food101",split = "train[:5000]")
food = food.train_test_split(test_size = 0.2)

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

Dataset food101 downloaded and prepared to /root/.cache/huggingface/datasets/food101/default/0.0.0/7cebe41a80fb2da3f08fcbef769c8874073a86346f7fb96dc0847d4dfc318295. Subsequent calls will reuse this data.


{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7F711B67B9D0>,
 'label': 10}

In [ ]:
print(food["train"][0])

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7F6FCE894B50>, 'label': 10}


In [ ]:
labels = food["train"].features["label"].names
label2id, id2label = dict(),dict()

for i,label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
id2label[str(79)]

'prime_rib'

In [ ]:
%%capture
'''
checkpoint = "google/cit-base-pathc16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
'''

In [ ]:
'''
normalize = Normalize(mean=image_processor.image_mean, std = image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)

transforms = Compose([RandomResizedCrop(size),ToTensor(),normalize])
'''

'\nnormalize = Normalize(mean=image_processor.image_mean, std = image_processor.image_std)\nsize = (\n    image_processor.size["shortest_edge"]\n    if "shortest_edge" in image_processor.size\n    else (image_processor.size["height"], image_processor.size["width"])\n)\n\ntransforms = Compose([RandomResizedCrop(size),ToTensor(),normalize])\n'

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples


In [ ]:
food = food.with_transform(transforms)

In [ ]:
data_collator = DefaultDataCollator()

## Tensorflow


In [ ]:
'''
size = (image_processor.size["height"], image_processor.size["widith"])
'''

'\nsize = (image_processor.size["height"], image_processor.size["widith"])\n'

In [ ]:
'''
train_data_augmentation = keras.Sequential([
    layers.RandomCrop(size[0],size[1]),
    layers.Rescaling(scale = 1.0/127.5,offset = -1),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(factor = 0.02),
    layers.RandomZoom(height_factor=0.2,width_factor=0.2)    
],
    name = "train_data_augmentation"
)
'''

'\ntrain_data_augmentation = keras.Sequential([\n    layers.RandomCrop(size[0],size[1]),\n    layers.Rescaling(scale = 1.0/127.5,offset = -1),\n    layers.RandomFlip("horizontal"),\n    layers.RandomRotation(factor = 0.02),\n    layers.RandomZoom(height_factor=0.2,width_factor=0.2)    \n],\n    name = "train_data_augmentation"\n)\n'

In [ ]:
'''
val_data_augmentation = keras.Sequential([
    layers.CenterCrop(size[0],size[1])
    layers.Rescaling(scale = 1.0/127.5,offset =-1)
],  
    name = "val_data_augmentation"
)
'''

'\nval_data_augmentation = keras.Sequential([\n    layers.CenterCrop(size[0],size[1])\n    layers.Rescaling(scale = 1.0/127.5,offset =-1)\n],  \n    name = "val_data_augmentation"\n)\n'

In [ ]:
def convert_to_tf_tensor(image,Image):
    np_image = np.array(image)
    tf_image = tf.convert_to_tensor(np_image)
    return tf.expand_dims(tf_image,0)
    

In [ ]:
def preprocess_val(example_batch):
    images = [
        val_data_augmentation(convert_to_tf_tensor(image.cnvert("RGB"))) for image in example_batch["image"]

    ]
    example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
    return example_batch

In [ ]:
def preprocess_train(example_batch):
    images = [
        train_data_augmentation(convert_to_tf_tensor(image.convert("RGB"))) for image in example_batch["image"]
        

        ]
    example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
    return example_batch


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions,axis = 1)
    return accuracy.compute(predictions=predictions,references = labels)
    

In [ ]:
model = AutoModelForImageClassification.from_pretrained()

In [ ]:
training_args = TrainingArguments()

In [ ]:
trainer = Trainer()

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
optimizer,lr_scheduler = create_optimizer()

In [ ]:
model = TFAutoModelForImageClassification()